# Apache Spark for astronomy: hands-on session 2

### Context

Welcome to the series of notebooks on Apache Spark! The main goal of this series is to get familiar with Apache Spark, and in particular its Python API called PySpark in a context of the astronomy. In this second notebook, we will learn on concrete examples how to interface and play with popular scientific libraries (Numpy, Pandas, ...).

### Learning objectives

- Interfacing popular Python scientific libraries with Apache Spark
- Developing your own modules for Spark

Through this series of exercises, we will use the same dataset as in the first session:

In [ ]:
# Load data into a Spark DataFrame
df = spark.read.format("parquet").load("../data/clusters.parquet")

## User defined functions and column creation

Similarly to `map` and `mapPartitions`, you would like to define your own functions but this time to create new DataFrame columns. In python, the efficient way of doing this is via "Pandas User Defined Functions" (vectorized functions). 

**Exercise (£):** Use pandas UDF to compute the distance of each row to the center (x, y, z) = (0, 0, 0), and store the result in a new Dataframe column:

**Exercise (£££):** As in session 1, find the barycentre of each clusters in the dataset but this time using aggregation and user defined function (hint: look for `PandasUDFType.GROUPED_MAP`). 